# visualize processing stages

Notebook to plot multiple mass channels at different stages of low-level processing for each point in each given slide.  
Each mass channel can be plotted against the background using red/blue comparison (bg in <span style="color:red;text-shadow:0px 0px 2px white">__red__</span>, mass channel in <span style="color:cyan;text-shadow:0px 0px 2px black">__blue__</span>, overlap in <span style="color:white;text-shadow:0px 0px 2px black">__white__</span> ).  
Images are saved as `.png` files within a directory structure headed at input `save_path`

Accepted format for input images: IonPath's __MIBITiff__ format from MIBI/O or MIBITracker

Please update the __input parameters__ as needed

## load headers

In [ ]:
# magic commands
%matplotlib notebook
%load_ext autoreload
%autoreload 2

#imports
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

import pathlib
import glob
import re

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec

from mibidata import mibi_image as mi, tiff
import visualize_data as viz

## input parameters

In [ ]:
data_path = pathlib.Path('../sample_data')
save_path = pathlib.Path('../sample_data/compOut/')

mass_channels = [89, 113, 146, 149, 159, 176, 115]

# background channel to compare to.  For no background comparison, set to -1
bg_channel = 197

# Option for comparing to bg channel
# pixels w/ non zero values -> 1.0 (shows all overlaps equally)
binary = False

# Anonymized by default currently
# TODO: Read panel if anonymize is false, and use target names
#anonymize_targets = True
#panel_path = data_path.joinpath('Panel54_MelanomaCohort_2.csv')

# slide foldernames for input folders
slide_foldernames = [
    'CohortSlide1'
]

# the above generally looks like:
#slide_foldernames = [
#    '191204_CohortSlide9_S14-47859-A2-S14-20352-A8',
#    '191205_CohortSlide7_S146936A2_G1471023A1_run',
#    '191206_CohortSlide11_TMA443_run',
#    '191211_CohortSlide10_S158066A1_SP153393_G1675892_run'
#]

# slide designations for output folders
# leave empty to use full slide name
slide_names = [
    'Sdemo'
]

# the above generally looks like:
#slide_names = [
#    'S9',
#    'S7',
#    'S11',
#    'S10'
#]

process_folder_prefix = ''
process_folder_suffix = '_TIFF'

raw_tiff_folder_name = 'bg_none'
proccessed_tiff_folder_name = 'bg_au_050_ta_020'

# process 'prefix' is PointN
process_suffix = '_RowNumber0_Depth_Profile0'
isobar_suffix = '-MassCorrected'
denoise_suffix = '-Filtered'

# Show only given mass channels
show_raw = mass_channels
show_bg_cor = mass_channels
show_iso = []
show_filter = mass_channels

## plot images

In [ ]:
plt.ioff()
plt.style.use('dark_background')

# show mask matrix
show_m = np.zeros((4,len(mass_channels)))
show_m[0,:] = np.isin(mass_channels, show_raw)
show_m[1,:] = np.isin(mass_channels, show_bg_cor)
show_m[2,:] = np.isin(mass_channels, show_iso)
show_m[3,:] = np.isin(mass_channels, show_filter)
show_m = np.ma.make_mask(show_m, shrink=False)

# use full slide name
if len(slide_names)==0:
    slide_names = slide_foldernames;

def plot_text(ax,text,fontsize):
    ax.text(0.5,0.5,text,verticalalignment='center',horizontalalignment='center',transform=ax.transAxes, color='yellow', fontsize=fontsize)
    ax.set_frame_on(False)
    ax.xaxis.set_visible(False)
    ax.yaxis.set_visible(False)

def make_fig(tiffs,slide,point,channel,titles):
    ncols = len(tiffs)+1
    
    plt.subplots_adjust(hspace=0.0,wspace=0.0)
    wr = [1] + ([3] * (ncols-1))
    gs = gridspec.GridSpec(2,ncols,height_ratios = [1,3],width_ratios=wr, hspace=0)
    
    for n, cell in enumerate(gs):
        ax = plt.subplot(cell)
        if n//ncols == 0:
            plot_text(ax,titles[n],14)
        else:
            if n%ncols==0:
                plot_text(ax,str(channel),14)
            else:
                print(str(tiffs[n%ncols-1]))
                image = tiff.read(str(tiffs[n%ncols-1]))
                im = image[channel]
                if bg_channel<0:
                    viz.plot_image(im, ax=ax,brighten_image=True,hi_res=True)
                else:
                    viz.plot_rb_image(image[bg_channel],im,ax=ax,binary=binary,normalize=True,brighten_image=True,hi_res=True)
        ax.xaxis.set_visible(False)
        ax.yaxis.set_visible(False)

    if not save_path.is_dir():
        save_path.mkdir()
    comp_dir_name = f'{slide}-{point}'
    comp_dir_path = save_path.joinpath(comp_dir_name)
    if not comp_dir_path.is_dir():
        comp_dir_path.mkdir()
    
    plt.gcf().set_figheight((1024*4/3)/300)
    plt.gcf().set_figwidth((1024*(ncols-2/3))/300)
    
    # save as .png for lossless compression
    plt.subplots_adjust(hspace=0.0,wspace=0.0)
    plt.savefig(str(comp_dir_path.joinpath(f'{channel}_binary_{binary}.png')), bbox_inches='tight', dpi=300, pad_inches=0.0)
    
    

for slide_n, slide in enumerate(slide_foldernames):
    tiffpath = data_path.joinpath(slide).joinpath(process_folder_prefix+slide+process_folder_suffix)
    rawpath = tiffpath.joinpath(raw_tiff_folder_name)
    print(f'Looking for TIFF files in {rawpath}')
    for rawfilename in glob.glob(f'{str(rawpath.absolute())}/*{process_suffix}.tiff'):
        pointNum = int(re.search(r'\d+',pathlib.Path(rawfilename).name).group())
        proc_tiffpath = tiffpath.joinpath(proccessed_tiff_folder_name)
        
        all_tiffs = np.array([
            rawpath.joinpath(rawfilename),
            proc_tiffpath.joinpath(f'Point{pointNum}{process_suffix}.tiff'),
            proc_tiffpath.joinpath(f'Point{pointNum}{process_suffix}{isobar_suffix}.tiff'),
            proc_tiffpath.joinpath(f'Point{pointNum}{process_suffix}{isobar_suffix}{denoise_suffix}.tiff')
        ])
        
        all_titles = np.array([
            'Raw Image',
            'Background Removed',
            'Isobaric Correction',
            'Denoised'
        ])
        
        for n_chan, channel in enumerate(mass_channels):

            make_fig(all_tiffs[show_m[:,n_chan]],slide_names[slide_n],pointNum,channel,np.concatenate((['Mass Channel'], all_titles[show_m[:,n_chan]])))
            
            #break #debug
            
        #break #debug
    #break #debug
print(f'Output PNG files saved in {save_path}')